In [5]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

import timeit
start = timeit.default_timer()

randomlist = []
demand = {}
random.seed(11) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(11) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,20))).astype(float)

random.seed(11) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(11) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(11) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()
cols = sorted(demand.keys())

In [6]:
costs1=copy.deepcopy(costs)
mi = 10000
# print(supply['S1'])
# break
# costs1=copy.deepcopy(costs)

# costs2=copy.deepcopy(costs)
# costs3=copy.deepcopy(costs)
# for i in supply:
#     mi=min(costs[i].values())
#     # print(costs[i])
#     # print(mi)
#     for j in costs2[i]:
#         costs2[i][j]-=mi
# # print(costs2)
# for i in demand :
#     mi=10000
#     for j in supply:
#         if costs[j][i]<mi :
#             mi=costs[j][i]
#     for j in supply:
#         costs3[j][i]=costs3[j][i]-mi
# # print(costs3)

# for i in demand:
#     for j in supply:
#         costs[j][i]= costs2[j][i]+costs3[j][i]


res = dict((k, defaultdict(int)) for k in costs)
g = {}
for x in supply:
    g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
for x in demand:
    g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])
# print(costs1)
flag=0
while g:

    d = {}
    # print(demand,supply)

    for x in demand:
        d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x])* costs[g[x][0]][x] if len(g[x]) > 1 else (costs[g[x][0]][x])*costs[g[x][0]][x]
    s = {}
    for x in supply:
        d[x] = (costs[x][g[x][1]] - costs[x][g[x][0]])* costs[x][g[x][0]] if len(g[x]) > 1 else costs[x][g[x][0]]*costs[x][g[x][0]]

    for x in supply :
        for y in demand:
            if costs[x][y]*min(supply[x],demand[y])<mi :
                mi= costs[x][y]*min(supply[x],demand[y])

    sd = max(d, key=lambda n: d[n])
    # t = max(s, key=lambda n: s[n])
    t,f="D","S"
    if demand.get(sd)!=None:
        t,f = sd, g[sd][0]
    else:
        t,f = g[sd][0],sd




    # t,f=tfin,fin
    # print(t,f)
    v = min(supply[f], demand[t])
    res[f][t] += v
    demand[t] -= v

    if demand[t] == 0:
        for k, n in supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del demand[t]
    supply[f] -= v
    if supply[f] == 0:
        for k, n in demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del supply[f]


# print(costs1)
cost = 0
for g in sorted(costs1):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")
# print ("Total Cost = ", cost)


In [7]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20
0,265.089476,110.045903,570.959014,809.125883,507.227612,576.273643,84.611746,574.439733,1033.696049,951.054976,795.568166,186.276546,958.946531,958.232439,261.995349,695.557029,132.172935,227.326598,410.941570,250.944259
1,952.534778,1025.520707,503.627977,451.250336,217.156315,189.537341,886.404746,773.702589,159.414533,651.206837,527.181276,1046.573873,887.547405,791.463743,729.001967,1148.811094,1238.669720,498.162916,370.665029,1003.406036
2,147.970039,450.352413,177.815497,400.987246,157.352142,378.348722,851.544758,981.705794,876.293338,656.649181,675.200796,434.442519,233.647218,533.539135,820.453300,126.656668,132.066304,871.251048,391.237859,785.310507
3,859.217198,1073.241173,530.899801,463.879908,937.667678,897.358068,759.466340,942.502040,770.781060,1110.730380,419.449249,855.541574,1067.604368,455.740771,741.768845,450.303718,598.227508,190.504689,116.379413,893.670903
4,640.873158,853.896708,1015.113950,599.907291,539.692166,301.201130,1043.971473,1108.152634,993.391063,1173.800195,861.409745,930.491409,305.655259,1043.182587,508.027594,602.278098,503.891359,807.549737,902.385677,798.964957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,253.087077,159.742502,693.866325,165.544716,666.964077,730.357563,647.324100,454.926875,411.296286,309.185773,768.099815,701.436888,238.621212,568.410302,759.097986,929.123137,448.574919,384.140574,975.210813,606.373425
2036,610.202594,988.584230,986.638013,926.179261,1079.876357,453.320192,818.383782,965.686771,663.066529,714.975059,818.211253,718.690918,480.078406,550.389839,480.645906,1161.015324,884.624709,1104.123942,578.693753,349.134284
2037,194.149405,838.167467,1091.031501,1014.857923,307.664192,913.672606,491.362221,95.689299,926.697452,775.638736,685.001779,181.698157,770.137205,544.313298,998.551646,948.010739,1014.818400,218.254877,1055.631212,1131.234590
2038,419.804996,449.483048,1317.984872,729.640325,1257.658741,289.539975,689.866918,360.331791,1203.739012,1243.655214,622.664840,992.110184,801.265449,808.906075,159.852832,258.778285,1175.996833,548.168201,295.377245,126.975769


In [8]:
cost

297722.85127478914